# Определение тематики текста

<p>В рамках данного проекта решается задача определения тематики текста.</p>
<p>Используется предобученная модель BERT.</p>
<p>Дообучение модели производится на размеченных текстах, которые представляют собой предложения из новостей различных тематик.</p>
<p>Число текстов - 5740, число тем - 14.</p>

### Загрузка библиотек

In [1]:
import pandas as pd
import numpy as np

import time

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

from sklearn.model_selection import train_test_split

### Объявление констант

In [2]:
# Начальное значение генератора случайных чисел
RANDOM_STATE = 12345

# Размер тестовой выборки
TEST_SIZE = 0.25

# Размер валидационной выборки
VALIDATION_SIZE = 0.2

# Число эпох обучения
EPOCHS = 10

### Загрузка данных

In [3]:
# Загружаем данные из файла
data = pd.read_csv('news/texts.csv', sep=';')

# Удаляем строки с пропусками
data = data.dropna()

### Исследовательский анализ и предобработка

In [4]:
# Выведем информацию о датафрейме
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5740 entries, 0 to 5742
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   topic   5740 non-null   float64
 1   text    5740 non-null   object 
dtypes: float64(1), object(1)
memory usage: 134.5+ KB


<p>Заменим тип столбца с метками классов на int.</p>

In [5]:
# Преобразовываем таргет в int
data['topic'] = data['topic'].astype(int)

In [6]:
# Выведем число пропусков
data.isna().sum()

topic    0
text     0
dtype: int64

<p>Пропусков не обнаружено.</p>
<p>Пррверим датафрейм на наличие дубликатов.</p>

In [7]:
# Выведем число дубликатов
data.duplicated().sum()

176

<p>Обнаружено 176 дубликатов, произведем их удаление.</p>

In [8]:
# Удалим дубликаты
data = data.drop_duplicates()

<p>Выведем список уникальных классов и их число.</p>

In [9]:
# Выведем уникальные классы (темы)
print('Уникальные классы:', sorted(data['topic'].unique()))

# Выведем число уникальных классов
print('Число классов:', len(data['topic'].unique()))

Уникальные классы: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 99]
Число классов: 14


<p>Создадим словарь с текстовыми метками классов.</p>

In [10]:
# Словарь с темами
themes = {
    0: 'Экономика',
    1: 'Финансы',
    2: 'Акции',
    3: 'Компании',
    4: 'Технологии',
    5: 'Политика',
    6: 'Наука',
    7: 'Здоровье',
    8: 'Энергетика',
    9: 'Происшествия',
    10: 'Спорт',
    15: 'Общество',
    20: 'Климат',
    99: 'Прочее'
}

In [11]:
# Создадим столбец с текстовым описанием классов
data['topic_text'] = data['topic'].apply(lambda x: themes[x])

# Вычислим число текстов в каждой категории
data['topic_text'].value_counts()

topic_text
Политика        3402
Компании         666
Экономика        404
Энергетика       396
Происшествия     216
Акции            152
Финансы          143
Технологии        73
Прочее            44
Наука             31
Спорт             26
Общество           8
Здоровье           2
Климат             1
Name: count, dtype: int64

<p>В наибольшей степени представлены новости из категорий 'Политика', 'Компании', 'Экономика', 'Энергетика'.</p>
<p>В остальных категориях число примеров гораздо меньше, что может отрицательно повлиять на обучение и качество предсказания данных категорий.</p>

In [12]:
# Создаем экземпляр LabelEncoder
encoder = LabelEncoder()

# Применим кодирование
data['encoded_topic'] = encoder.fit_transform(data['topic'])

### Создание выборок

In [13]:
# Список с текстами
texts = list(data['text'])

# Список с категориями
labels = list(data['encoded_topic'])

In [14]:
# Обучающая и тестовая выбоки
X_train, X_test, y_train, y_test = train_test_split(
    texts,
    labels,
    test_size=TEST_SIZE,
    shuffle=True,
    random_state=RANDOM_STATE)

# Дополнительное разделение обучающей выборки на обучающую и валидационную
X_train, X_val, y_train, y_val = train_test_split(
    X_train,
    y_train,
    test_size=VALIDATION_SIZE,
    shuffle=True,
    random_state=RANDOM_STATE
)

### Загрузка модели

In [15]:
# Число классов
num_labels = len(encoder.classes_)

In [16]:
# Загрузка предварительно обученной модели DistilBERT
model_name = "distilbert-base-multilingual-cased"

# Загрузка токенизатора
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Создание датасетов и загрузчиков

In [17]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt",
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels": torch.tensor(label),
        }

In [18]:
# Подготовка данных
max_length = 128

# Обучающий датасет и загрузчик
dataset = TextDataset(X_train, y_train, tokenizer, max_length)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Валидационный датасет и загрузчик
dataset = TextDataset(X_val, y_val, tokenizer, max_length)
val_dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Тестовый датасет и загрузчик
dataset = TextDataset(X_test, y_test, tokenizer, max_length)
test_dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

### Fine-tuning модели

In [19]:
# Функция потерь
criterion = nn.CrossEntropyLoss()

# Начальное значение потерь
avg_train_loss = float('inf')

In [20]:
%%time

# Обучение модели
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

model.train()
for epoch in range(EPOCHS):  # Количество эпох обучения
    # Старт таймера
    start_time = time.time()
    
    train_losses = []
    len_batch = len(dataloader)
    for i, batch in enumerate(dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        train_losses.append(loss.item())
        avg_train_loss = np.mean(train_losses)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Валидация
    val_losses = []
    predicted_labels = []
    true_labels = []
    with torch.no_grad():
        for batch in val_dataloader:
            val_targets = batch["labels"]
            batch = {
            "input_ids": batch["input_ids"].to(device),
            "attention_mask": batch["attention_mask"].to(device)
            }
            val_outputs = model(**batch)
            val_loss = criterion(val_outputs.logits, val_targets)
            val_losses.append(val_loss.item())

            # Получение предсказанных меток
            _, preds = torch.max(val_outputs.logits, dim=1)
            predicted_labels.extend(preds.cpu().numpy())
            true_labels.extend(val_targets.cpu().numpy())

    avg_val_loss = np.mean(val_losses)

    accuracy = accuracy_score(true_labels, predicted_labels)
    t = round((time.time() - start_time)/60, 2)
    print(f"Epoch {epoch+1}, Train Loss: {round(avg_train_loss.item(),4)}, Val Loss: {round(avg_val_loss, 4)}, Accuracy: {round(accuracy, 4)}, t={t}")

Epoch 1, Train Loss: 1.3565, Val Loss: 0.9545, Accuracy: 0.7329, t=5.93
Epoch 2, Train Loss: 0.7585, Val Loss: 0.7264, Accuracy: 0.794, t=5.25
Epoch 3, Train Loss: 0.5018, Val Loss: 0.636, Accuracy: 0.8168, t=5.26
Epoch 4, Train Loss: 0.3531, Val Loss: 0.5737, Accuracy: 0.8251, t=5.25
Epoch 5, Train Loss: 0.2513, Val Loss: 0.5949, Accuracy: 0.8431, t=5.19
Epoch 6, Train Loss: 0.191, Val Loss: 0.6041, Accuracy: 0.8395, t=5.14
Epoch 7, Train Loss: 0.1451, Val Loss: 0.65, Accuracy: 0.8311, t=5.22
Epoch 8, Train Loss: 0.1241, Val Loss: 0.5235, Accuracy: 0.8611, t=5.28
Epoch 9, Train Loss: 0.0854, Val Loss: 0.576, Accuracy: 0.8503, t=5.28
Epoch 10, Train Loss: 0.0603, Val Loss: 0.5783, Accuracy: 0.8623, t=5.28
CPU times: user 59min 10s, sys: 33min 42s, total: 1h 32min 52s
Wall time: 53min 5s


### Проверка на тестовых данных

In [21]:
# Тестирование модели
model.eval()
test_losses = []
predicted_labels = []
true_labels = []
with torch.no_grad():
    for batch in test_dataloader:
        test_targets = batch["labels"]
        batch = {
            "input_ids": batch["input_ids"].to(device),
            "attention_mask": batch["attention_mask"].to(device)
        }
        test_outputs = model(**batch)
        test_loss = criterion(test_outputs.logits, test_targets)
        test_losses.append(test_loss.item())

        # Получение предсказанных меток
        _, preds = torch.max(test_outputs.logits, dim=1)
        predicted_labels.extend(preds.cpu().numpy())
        true_labels.extend(test_targets.cpu().numpy())

avg_test_loss = np.mean(test_losses)
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Test Loss: {round(avg_test_loss, 4)}, Accuracy: {round(accuracy, 4)}")

Test Loss: 0.5174, Accuracy: 0.8728


### Инференс

<p>Подготовим функцию для предсказания темы текста при помощи обученной модели.</p>

In [22]:
# Функция предсказывает тематику текста
def predict_class(text):
    
    # Токенизация и преобразование текста в тензор
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    # Получение предсказания
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Преобразование логитов в вероятности
    probs = torch.softmax(logits, dim=1)

    # Определение класса с наибольшей вероятностью
    _, predicted_class = torch.max(probs, dim=1)

    return predicted_class.item()

<p>Создадим новые предложения различных тематик для проверки работы модели.</p>

In [32]:
# Пример текста для классификации
texts = []
texts.append('Президент Бразилии рассчитывает на урегулирование через переговоры.')
texts.append('Илон Маск купил сервис микроблогов в прошлом году. Сумма сделки составила $44 млрд. Сделка была объявлена в апреле, однако закрыта была в конце октября после длительных переговором между сторонами.')
texts.append('Промышленный индекс ISM в США в июне неожиданно опустился ниже 50 пунктов.')
texts.append('В середине торгов цена Brent перешла к росту, подорожав до 80 долл. за баррель.')
texts.append('Рост ВВП Франции в 2025 г. составил 1,6%.')
texts.append('Объем денежных перевод из-за рубежа в Грузию в 2023 г. составил 4,1 млрд долл. (-6,8% к предыдущему году). Поступления из РФ упали на 27,5% (1,5 млрд долл.).')
texts.append('Nasdaq уведомила Qiwi plc о делистинге ее американских депозитарных акций. Qiwi намерена обжаловать это решение.')
texts.append('Футболисты сборной Англии победили Словакию со счетом 4:2 в матче чемпионата Европы, а Испания обыграла Грузию (2:0).')

<p>Выведем предсказанные тематики текстов и сами тексты.</p>

In [33]:
%time
# Выполним предсказание на новых примерах
for text in texts:
    print(themes[predict_class(text)], '-', text)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 7.15 µs
Политика - Президент Бразилии рассчитывает на урегулирование через переговоры.
Компании - Илон Маск купил сервис микроблогов в прошлом году. Сумма сделки составила $44 млрд. Сделка была объявлена в апреле, однако закрыта была в конце октября после длительных переговором между сторонами.
Экономика - Промышленный индекс ISM в США в июне неожиданно опустился ниже 50 пунктов.
Энергетика - В середине торгов цена Brent перешла к росту, подорожав до 80 долл. за баррель.
Экономика - Рост ВВП Франции в 2025 г. составил 1,6%.
Финансы - Объем денежных перевод из-за рубежа в Грузию в 2023 г. составил 4,1 млрд долл. (-6,8% к предыдущему году). Поступления из РФ упали на 27,5% (1,5 млрд долл.).
Акции - Nasdaq уведомила Qiwi plc о делистинге ее американских депозитарных акций. Qiwi намерена обжаловать это решение.
Спорт - Футболисты сборной Англии победили Словакию со счетом 4:2 в матче чемпионата Европы, а Испания обыграла Грузию (2

<p>Модель верно определила тематики текстов, включая минорные классы (например, спорт).</p>

### Выводы

<p>В ходе работы над проектом было сделано:</p>
<ul>
    <li>Загружены необходимые библиотеки.</li>
    <li>Объявлены константы.</li>
    <li>Загружены данные.</li>
    <li>Проведены предобработка и исследовательский анализ данных:<br>
        - тип данных в столбце topic изменен на int;<br>
        - проведена проверка на пропуски (пропуски не обнаружены);<br>
        - обнаружены и удалены дубликаты;<br>
        - подготовлен словарь с текстовыми обозначениями классов;<br>
        - проанализирована частотность классов в датафрейме (наиболее часто встречаются 'Политика', 'Компании', 'Экономика', 'Энергетика');<br>
        - закодирован таргет.</li>
    <li>Данные разделены на выборки: обучающую, валидационную и тестовую.</li>
    <li>Загружена модель BERT и токенизатор.</li>
    <li>Созданы датасеты и загрузчики.</li>
    <li>Произведено дообучение нейросети (модели BERT). Значение метрики Accuracy на валидации составило 0,8623.</li>
    <li>Произведена проверка модели на тестовых данных. Значение Accuracy на тесте равно 0,8728.</li>
    <li>Проведена проверка модели на новых текстах. Для 8 текстов разных тематик предсказания классов выполнены верно.</li>
</ul>

<p>Возможные доработки модели:</p>
<ul>
    <li>Увеличение объема обучающих данных. Для этого потребуется разметка новых текстов.</li>
    <li>Применение коэффициента l2_lambda для предотвращения переобучения модели.</li>
    <li>Подбор гиперпараметров при помощи Optuna: шага обучения и коэффициента l2_lambda.</li>
    <li>Объединение низкочастотных тематик, близких по содержанию (наука, технологии).</li>
</ul>

<p>Практическая применимость модели:</p>
<ul>
    <li>После проведения доработок модель может быть использована для определения тематик новостей.</li>
<ul>